In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)
df.head(2)


[Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date='2022-04-08', date_built='2016', price='936923', bedrooms='4', bathrooms='3', sqft_living='3167', sqft_lot='11733', floors='2', waterfront='1', view='76'),
 Row(id='7530a2d8-1ae3-4517-9f4a-befe060c4353', date='2021-06-13', date_built='2013', price='379628', bedrooms='2', bathrooms='2', sqft_living='2235', sqft_lot='14384', floors='1', waterfront='0', view='23')]

In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [9]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = '''
  SELECT ROUND(AVG(price), 2) FROM home_sales WHERE bedrooms = 4
'''
spark.sql(query).show()

+--------------------+
|round(avg(price), 2)|
+--------------------+
|           299661.01|
+--------------------+



In [13]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = '''
  SELECT ROUND(AVG(price), 2), YEAR(date) AS yr FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY yr ORDER BY yr DESC
'''
spark.sql(query).show()


+--------------------+----+
|round(avg(price), 2)|  yr|
+--------------------+----+
|           292725.69|2022|
|           294211.46|2021|
|           294204.16|2020|
|           287287.82|2019|
+--------------------+----+



In [14]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = '''
  SELECT ROUND(AVG(price), 2), YEAR(date) as yr FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY yr ORDER BY yr DESC
'''
spark.sql(query).show()

+--------------------+----+
|round(avg(price), 2)|  yr|
+--------------------+----+
|           290242.99|2022|
|           296330.96|2021|
|           292289.09|2020|
|           289859.14|2019|
+--------------------+----+



In [27]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = '''
  SELECT ROUND(AVG(price), 2) AS avg_price, view FROM home_sales
  GROUP BY view
  HAVING AVG(price) > 350000
  ORDER BY view DESC
'''
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| avg_price|view|
+----------+----+
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
| 991767.38|  80|
+----------+----+
only showing top 20 rows

--- 0.9688577651977539 seconds ---


In [28]:
# 7. Cache the the temporary table home_sales.
spark.sql('CACHE TABLE home_sales')

DataFrame[]

In [29]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [30]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----------+----+
| avg_price|view|
+----------+----+
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
| 991767.38|  80|
+----------+----+
only showing top 20 rows

--- 0.5782794952392578 seconds ---


In [31]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('home_sales_partition')

In [32]:
# 11. Read the parquet formatted data.
df_p = spark.read.parquet('home_sales_partition')

In [33]:
# 12. Create a temporary table for the parquet data.
df_p.createOrReplaceTempView('home_sales_p')

In [34]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
query = '''
  SELECT ROUND(AVG(price), 2) AS avg_price, view FROM home_sales_p
  GROUP BY view
  HAVING AVG(price) > 350000
  ORDER BY view DESC
'''
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| avg_price|view|
+----------+----+
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
| 991767.38|  80|
+----------+----+
only showing top 20 rows

--- 1.0505609512329102 seconds ---


In [36]:
# 14. Uncache the home_sales temporary table.
spark.sql('UNCACHE TABLE home_sales')

DataFrame[]

In [37]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False